# LU Factorization with Partial Pivoting

Copyright (C) 2021 Andreas Kloeckner

<details>
<summary>MIT License</summary>
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
</details>

In [70]:
import numpy as np
import numpy.linalg as la   
np.set_printoptions(linewidth=150, suppress=True, precision=3)

In [ ]:
n = 5

A = np.array([[2, 4, -2], [4, 9, -3], [-2, -3, 7]])
# A[0,0] = 0
A

array([[ 2,  4, -2],
       [ 4,  9, -3],
       [-2, -3,  7]])

## Permutation Matrices
This function returns a matrix that swaps rows `i` and `j`:

In [72]:
def row_swap_mat(i, j):
    P = np.eye(3)
    P[i] = 0
    P[j] = 0
    P[i, j] = 1
    P[j, i] = 1
    return P

## Pivoted LU: Initialization

We're trying to obtain $PA=LU$. Initialize:

In [73]:
i = 0
I = np.eye(3)
P = I.copy()
Lsub = np.zeros_like(A)
U = np.zeros_like(A)
remaining = A.copy()
remaining

array([[ 2,  4, -2],
       [ 4,  9, -3],
       [-2, -3,  7]])

## First column

First, find the pivot as `ipiv`:

In [74]:
#clear
ipiv = i + np.argmax(np.abs(remaining[i:, i]))
ipiv

1

Swap the rows in `remaining`, record in `P`:

In [75]:
#clear
swap_mat = row_swap_mat(i, ipiv)
print(swap_mat)
P = swap_mat @ P
print(P)
remaining = swap_mat @ remaining
remaining

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


array([[ 4.,  9., -3.],
       [ 2.,  4., -2.],
       [-2., -3.,  7.]])

Now carry out a step of LU, as above:

In [77]:

U[i, i:] = remaining[i, i:]
Lsub[i+1:,i] = remaining[i+1:,i]/ U[i,i]
remaining[i+1:, i+1:] -= np.outer(Lsub[i+1:,i], U[i, i+1:])
val = np.outer(Lsub[i+1:,i], U[i, i+1:])
print(val)
print(remaining)
# print(P@A-(Lsub+I)@U)

[[0 0]
 [0 0]]
[[ 4.  9. -3.]
 [ 2.  4. -2.]
 [-2. -3.  7.]]


## Subsequent columns

Find the pivot and perform the swaps so that you still have a valid $PA=LU$ factorization:

In [34]:
#clear
ipiv = i + np.argmax(np.abs(remaining[i:, i]))
swap_mat = row_swap_mat(i, ipiv)

P = swap_mat @ P
Lsub = swap_mat @ Lsub

# We need to similarly swap the rows in the L_sub here.

remaining = swap_mat @ remaining

print(ipiv)

1


Here are some checks to make sure you're on the right track:

In [35]:
print("Should maintain the same 'zero fringe' as the previous step:")
print(P @ A - (Lsub+I) @ U)

print("Should be zero:")
print(remaining[i:, i:] - (P @ A - (Lsub+I) @ U)[i:, i:])

Should maintain the same 'zero fringe' as the previous step:
[[ 0.  0.  0.]
 [ 2.  4. -2.]
 [-2. -3.  7.]]
Should be zero:
[[0. 0.]
 [0. 0.]]


Carry out a step of LU, as always:

In [ ]:
# U[i, i:] = remaining[i, i:]
# Lsub[i+1:,i] = remaining[i+1:,i]/U[i,i]
# remaining[i+1:, i+1:] -= np.outer(Lsub[i+1:,i], U[i, i+1:])
val = np.outer(Lsub[i+1:,i], U[i, i+1:])
print(val)
# i = i + 1
# remaining = remaining - np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
print(remaining)
print(P@A-(Lsub+I)@U)

[[ 3.  9. -3.]
 [ 2.  3. -2.]
 [-2. -3.  6.]]
[[ 0.  0.  0.]
 [ 2.  4. -2.]
 [-2. -3.  7.]]


## Inspect the result

In [39]:
P

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [40]:
I+Lsub

array([[ 1.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.571,  1.   ,  0.   ,  0.   ,  0.   ],
       [ 0.856,  0.515,  1.   ,  0.   ,  0.   ],
       [ 0.   , -0.508,  0.   ,  1.   ,  0.   ],
       [ 0.021,  0.627,  0.   ,  0.   ,  1.   ]])

In [ ]:
U

## Questions

Why do we switch to maintaining `Lsub` instead of all of `L`?

<details>
Realize that <code>P</code> is a permutation of the "trailing" rows. Applying to <code>Lsub</code> avoids disturbing the diagonal.
</details>

---

Why is it mathematically correct to only apply the permutations to `Lsub` and not all of `L`?

<details>
Realize that <code>P</code> is a permutation of the "trailing" rows. If we applied to <code>Lsub + I</code> instead, then the pivots would need to go into the row to which the 1 from the diagonal was permuted. Instead, it goes into <code>U[i,i]</code>, and leaving the diagonal of <code>L</code> undisturbed reflects that.
</details>

In [81]:
import numpy as np
np.set_printoptions(linewidth=150, suppress=True, precision=3)

A = np.array([[2., 4., -2.],
              [4., 9., -3.],
              [-2., -3., 7.]])

n = A.shape[0]
remaining = A.copy()
U = np.zeros_like(A)
Lsub = np.eye(n)
P = np.eye(n)

def row_swap_mat(i, j, n=3):
    P = np.eye(n)
    if i != j:
        P[[i, j]] = P[[j, i]]
    return P

for i in range(n):
    ipiv = i + np.argmax(np.abs(remaining[i:, i]))
    swap_mat = row_swap_mat(i, ipiv, n)
    remaining = swap_mat @ remaining
    P = swap_mat @ P
    
    # Only swap L's *previous columns* (not full matrix!)
    if i > 0:
        Lsub[[i, ipiv], :i] = Lsub[[ipiv, i], :i]
    
    print(f"\n--- Iteration {i} ---")
    print("Pivot:", ipiv)
    print("Remaining after swap:\n", remaining)
    
    U[i, i:] = remaining[i, i:]
    Lsub[i+1:, i] = remaining[i+1:, i] / U[i, i]
    
    outer = np.outer(Lsub[i+1:, i], U[i, i+1:])
    print("Outer product:\n", outer)
    
    remaining[i+1:, i+1:] -= outer
    print("Remaining after update:\n", remaining)

print("\nFinal matrices:")
print("P =\n", P)
print("L =\n", Lsub)
print("U =\n", U)
print("\nCheck: P@A =\n", P@A)
print("L@U =\n", Lsub@U)



--- Iteration 0 ---
Pivot: 1
Remaining after swap:
 [[ 4.  9. -3.]
 [ 2.  4. -2.]
 [-2. -3.  7.]]
Outer product:
 [[ 4.5 -1.5]
 [-4.5  1.5]]
Remaining after update:
 [[ 4.   9.  -3. ]
 [ 2.  -0.5 -0.5]
 [-2.   1.5  5.5]]

--- Iteration 1 ---
Pivot: 2
Remaining after swap:
 [[ 4.   9.  -3. ]
 [-2.   1.5  5.5]
 [ 2.  -0.5 -0.5]]
Outer product:
 [[-1.833]]
Remaining after update:
 [[ 4.     9.    -3.   ]
 [-2.     1.5    5.5  ]
 [ 2.    -0.5    1.333]]

--- Iteration 2 ---
Pivot: 2
Remaining after swap:
 [[ 4.     9.    -3.   ]
 [-2.     1.5    5.5  ]
 [ 2.    -0.5    1.333]]
Outer product:
 []
Remaining after update:
 [[ 4.     9.    -3.   ]
 [-2.     1.5    5.5  ]
 [ 2.    -0.5    1.333]]

Final matrices:
P =
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
L =
 [[ 1.     0.     0.   ]
 [-0.5    1.     0.   ]
 [ 0.5   -0.333  1.   ]]
U =
 [[ 4.     9.    -3.   ]
 [ 0.     1.5    5.5  ]
 [ 0.     0.     1.333]]

Check: P@A =
 [[ 4.  9. -3.]
 [-2. -3.  7.]
 [ 2.  4. -2.]]
L@U =
 [[ 4.  9. -3.]
 [-2

In [ ]:
for i in range(n):
    ipiv = i + np.argmax(np.abs(remaining[i:, i]))
    swap_mat = row_swap_mat(i, ipiv, n)
    remaining = swap_mat @ remaining
    P = swap_mat @ P
    
    # Only swap L's *previous columns* (not full matrix!)
    if i > 0:
        Lsub[[i, ipiv], :i] = Lsub[[ipiv, i], :i]
    
    print(f"\n--- Iteration {i} ---")
    print("Pivot:", ipiv)
    print("Remaining after swap:\n", remaining)
    
    U[i, i:] = remaining[i, i:]
    Lsub[i+1:, i] = remaining[i+1:, i] / U[i, i]
    
    outer = np.outer(Lsub[i+1:, i], U[i, i+1:])
    print("Outer product:\n", outer)
    
    remaining[i+1:, i+1:] -= outer
    print("Remaining after update:\n", remaining)
